<a href="https://colab.research.google.com/github/Nidzoki/PMFSTIntroToDS/blob/main/w6/06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">

### 2025./2026.

# UVOD U PODATKOVNU ZNANOST
## **Domaća zadaća  6**
#### Zadaci za samostalan rad

</div>

## Podaci

*  2240 zapisa (kupaca) s 28 značajki vezanih za marketing
  * marketing_data.csv
* koristite stupce "ID", "Education", "Marital_Status", " Income "
* preimenujte stupac `Income` (jer sadrži razmake s kojima je nezgodno raditi)
* pretvorite stupac `Income` u brojčane vrijednosti (da možete računati srednju vrijednost)
* izbacite one retke koji imaju `NaN` vrijednost u bilo kojem od stupaca s kojima radimo


Vaš zadatak je obaviti testiranje hipoteza nad danim podacima, prateći upute u **Zadatku 1** i **Zadatku 2**.

# Zadatak 0.
Izračunajte prosječnu plaću kupaca koji:
* imaju fakultetsku diplomu (`Education in ["2n Cycle", "Master", "PhD"]`)
* nemaju fakultetsku diplomu (`Education in ["Basic", "Graduation"]`)

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_csv("/content/drive/MyDrive/marketing_data.csv")
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Response,Complain,Country
0,1826,1970,Graduation,Divorced,"$84,835.00",0,0,6/16/14,0,189,...,6,1,0,0,0,0,0,1,0,SP
1,1,1961,Graduation,Single,"$57,091.00",0,0,6/15/14,0,464,...,7,5,0,0,0,0,1,1,0,CA
2,10476,1958,Graduation,Married,"$67,267.00",0,1,5/13/14,0,134,...,5,2,0,0,0,0,0,0,0,US
3,1386,1967,Graduation,Together,"$32,474.00",1,1,5/11/14,0,10,...,2,7,0,0,0,0,0,0,0,AUS
4,5371,1989,Graduation,Single,"$21,474.00",1,0,4/8/14,0,6,...,2,7,1,0,0,0,0,1,0,SP


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   2240 non-null   int64 
 1   Year_Birth           2240 non-null   int64 
 2   Education            2240 non-null   object
 3   Marital_Status       2240 non-null   object
 4    Income              2216 non-null   object
 5   Kidhome              2240 non-null   int64 
 6   Teenhome             2240 non-null   int64 
 7   Dt_Customer          2240 non-null   object
 8   Recency              2240 non-null   int64 
 9   MntWines             2240 non-null   int64 
 10  MntFruits            2240 non-null   int64 
 11  MntMeatProducts      2240 non-null   int64 
 12  MntFishProducts      2240 non-null   int64 
 13  MntSweetProducts     2240 non-null   int64 
 14  MntGoldProds         2240 non-null   int64 
 15  NumDealsPurchases    2240 non-null   int64 
 16  NumWeb

In [9]:
data.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', ' Income ',
       'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines',
       'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Response', 'Complain', 'Country'],
      dtype='object')

In [39]:
data[' Income '].head()

,Income
0,"$84,835.00"
1,"$57,091.00"
2,"$67,267.00"
3,"$32,474.00"
4,"$21,474.00"


In [41]:
data[' Income '].transform(lambda x : type(x)).value_counts()

,count
Income,
<class 'str'>,2216
<class 'float'>,24


#### Neki podaci su 'str', a neki 'float', moramo pretvoriti sve u float

In [50]:
data['income_transformed'] = data[' Income '].transform(lambda x : x if type(x) == float else float(x.replace('$', '').replace(',', '')))
data['income_transformed']


,income_transformed
count,2216.000000
mean,52247.251354
std,25173.076661
min,1730.000000
25%,35303.000000
50%,51381.500000
75%,68522.000000
max,666666.000000


In [51]:
diploma_income = data[data['Education'].isin(["2n Cycle", "Master", "PhD"])]['income_transformed']
no_diploma_income = data[data['Education'].isin(["Basic", "Graduation"])]['income_transformed']

In [55]:
print(f"Prosječna plaća kupaca koji imaju fakultetsku diplomu: ${round(diploma_income.mean(), 2)}")
print(f"Prosječna plaća kupaca koji nemaju fakultetsku diplomu: ${round(no_diploma_income.mean(), 2)}")

Prosječna plaća kupaca koji imaju fakultetsku diplomu: $53391.43
Prosječna plaća kupaca koji nemaju fakultetsku diplomu: $51224.34


## Zadatak 1.
Zanima nas:

> Je li prosječna plaća onih kupaca koji imaju fakultetsku diplomu (`Education in ["2n Cycle", "Master", "PhD"]`) statistički različita od kupaca koji nemaju fakultetsku diplomu (`Education in ["Basic", "Graduation"]`)?


* sročite hipoteze ($H_0$ i $H_1$)
* razina značajnosti testa: **$\alpha$ = 0.05**
* odaberite test koji ćete koristiti
  * hoće li biti parametrijski ili neparametrijski? (o čemu to ovisi?)
* donesite odluku - odbacujete li nultu hipotezu ili ne, te što to znači za postavljeno pitanje






1.  **Nul-Hipoteza ($H_0$):** nema statističke razlike između srednje vrijednosti dohotka kupaca sa fakultetskom diplomom i kupaca bez fakultetske diplome.
    $\\mu_{\text{diploma}} = \\mu_{\text{no diploma}}$

2.  **Alternativna Hipoteza ($H_1$):** postoji statistička razlika između srednje vrijednosti dohotka kupaca sa fakultetskom diplomom i kupaca bez fakultetske diplome. This is a two-tailed hypothesis.  
    $\\mu_{\text{diploma}} \\neq \\mu_{\text{no diploma}}$

In [59]:
alfa = 0.05
print(f"Razina značajnosti testa (alfa): {alfa}")

Razina značajnosti testa (alfa): 0.05


In [70]:
from scipy.stats import levene

# Perform Levene's test for homogeneity of variances
stat, p = levene(diploma_income.dropna(), no_diploma_income.dropna())

print(f"Levene's test statistic: {stat:.4f}")
print(f"Levene's p-value: {p:.4f}")

if p < alfa:
    print("Conclusion: p-value < alpha. Reject the null hypothesis of equal variances. Variances are not equal.")
else:
    print("Conclusion: p-value >= alpha. Fail to reject the null hypothesis of equal variances. Variances are equal.")

Levene's test statistic: 9.7939
Levene's p-value: 0.0018
Conclusion: p-value < alpha. Reject the null hypothesis of equal variances. Variances are not equal.


Po Levenovom testu, varijance nisu jednake (p-vrijednost < alfa). Zato ću provesti t-test na uzorcima sa pretpostavkom da su im varijance različite.

In [71]:
from scipy.stats import ttest_ind

stat, p_value = ttest_ind(diploma_income.dropna(), no_diploma_income.dropna(), equal_var=False)

print(f"t-test statistika: {stat:.4f}")
print(f"t-test p-vrijednost: {p_value:.4f}")


t-test statistika: 2.0581
t-test p-vrijednost: 0.0397


Da, prosječni prihod kupaca sa sveučilišnom diplomom statistički se razlikuje od prosječnog prihoda kupaca bez sveučilišne diplome.

*   Nulta hipoteza ($H_0$) tvrdila je da ne postoji statistička razlika u prosječnom prihodu između kupaca sa i bez sveučilišne diplome, dok je alternativna hipoteza ($H_1$) predlagala postojanje statističke razlike.
*   Razina značajnosti ($\alpha$) za testove hipoteza postavljena je na 0,05.
*   Leveneov test za homogenost varijanci dao je statistiku od 9,7939 i p-vrijednost od 0,0018. Budući da je ta p-vrijednost manja od $\alpha$ (0,05), nulta hipoteza o jednakim varijancama je odbačena, što ukazuje da varijance dviju skupina prihoda nisu jednake.
*   Proveden je t-test za nezavisne uzorke (Welchov t-test, zbog nejednakih varijanci), što je rezultiralo t-statistikom od 2,0581 i p-vrijednošću od 0,0397.
*   Budući da je p-vrijednost t-testa (0,0397) manja od $\alpha$ (0,05), nulta hipoteza o nepostojanju razlike u prosječnom prihodu je odbačena.

## Zadatak 2.
Zanima nas:  

> Postoji li statistički značajna razlika između prihoda i bračnog statusa?

* grupirajte bračni status u dvije kategorije - Alone (Divorced, Single, Absurd, Widow, YOLO) i In Couple (Married, Together)
* grupirajte prihode u 3 kategorije: kategorija **"A"** ako je prihod manji od ili jednak 45000, kategorija **"C"** ako je prihod veći od ili jednak 65000, a kategorija **"B"** ako je između tih vrijednosti
* [pandas.crosstab](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html)


* sročite hipoteze ($H_0$ i $H_1$)
> *hint*: hipoteze će govoriti o tome jesu li kategoričke varijable (prihodi i bračni status) zavisne ili nezavisne

* razina značajnosti testa: **$\alpha$ = 0.05**
* proučite [*Chi-squared test*](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) te ga iskoristite
> * [Chi-Square Test of Independence](https://www.scribbr.com/statistics/chi-square-test-of-independence/)
> * [primjer koda](https://machinelearningmastery.com/chi-squared-test-for-machine-learning/#:~:text=The%20Pearson's%20chi%2Dsquared%20test%20for%20independence%20can%20be%20calculated,for%20the%20two%20categorical%20variables.)
* donesite odluku - odbacujete li nultu hipotezu ili ne, te što to znači za postavljeno pitanje

In [78]:
alone_statuses = ["Divorced", "Single", "Absurd", "Widow", "YOLO", "Alone"]
in_couple_statuses = ["Married", "Together"]

data['marital_status_grouped'] = np.select(
    [data['Marital_Status'].isin(alone_statuses),
     data['Marital_Status'].isin(in_couple_statuses)],
    ["Alone", "In Couple"],
    default="Other"
)

print(data[['Marital_Status', 'marital_status_grouped']].value_counts().sort_index())

Marital_Status  marital_status_grouped
Absurd          Alone                       2
Alone           Alone                       3
Divorced        Alone                     232
Married         In Couple                 864
Single          Alone                     480
Together        In Couple                 580
Widow           Alone                      77
YOLO            Alone                       2
Name: count, dtype: int64


In [79]:
data['income_category'] = pd.cut(data['income_transformed'],
                                   bins=[0, 45000, 65000, np.inf],
                                   labels=['A', 'B', 'C'],
                                   right=True)

print(data[['income_transformed', 'income_category']].value_counts().sort_index())

income_transformed  income_category
1730.0              A                  1
2447.0              A                  1
3502.0              A                  1
4023.0              A                  1
4428.0              A                  1
                                      ..
157243.0            C                  1
157733.0            C                  1
160803.0            C                  1
162397.0            C                  1
666666.0            C                  1
Name: count, Length: 1974, dtype: int64


In [80]:
contingency_table = pd.crosstab(data['marital_status_grouped'], data['income_category'])
print(contingency_table)

income_category           A    B    C
marital_status_grouped               
Alone                   313  241  232
In Couple               587  399  444


## Formulacija Hipoteza za Chi-kvadrat test

**Nulta hipoteza ($H_0$):** Ne postoji statistički značajna povezanost (nezavisni su) između bračnog statusa (grupiranog u 'Alone' i 'In Couple') i kategorija prihoda ('A', 'B', 'C').

**Alternativna hipoteza ($H_1$):** Postoji statistički značajna povezanost (ovisni su) između bračnog statusa (grupiranog u 'Alone' i 'In Couple') i kategorija prihoda ('A', 'B', 'C').

Razina značajnosti je postavljena na **$\alpha = 0.05$**.

In [81]:
from scipy.stats import chi2_contingency

chi2_stat, p_value, dof, expected_freq = chi2_contingency(contingency_table)

print(f"Chi-squared Statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:\n", expected_freq)


Chi-squared Statistic: 1.9158
P-value: 0.3837
Degrees of Freedom: 2
Expected Frequencies:
 [[319.22382671 227.00361011 239.77256318]
 [580.77617329 412.99638989 436.22743682]]


## Odluka i interpretacija Chi-kvadrat testa

Nakon provedbe Chi-kvadrat testa, dobiveni su sljedeći rezultati:
*   **Chi-kvadrat statistika:** 1.9158
*   **P-vrijednost:** 0.3837
*   **Stupnjevi slobode:** 2

Postavljena razina značajnosti ($\alpha$) je **0.05**.

**Odluka:**
Budući da je p-vrijednost (0.3837) veća od razine značajnosti ($\alpha = 0.05$), **ne odbacujemo nultu hipotezu ($H_0$)**.

**Interpretacija:**
To znači da ne postoji statistički značajna povezanost između bračnog statusa (grupiranog u 'Alone' i 'In Couple') i kategorija prihoda ('A', 'B', 'C') na razini značajnosti od 0.05. Drugim riječima, na temelju ovih podataka, ne možemo zaključiti da su ove dvije varijable zavisne; one su statistički nezavisne. Prosječni prihod kupca ne ovisi o njegovom bračnom statusu.